In [1]:
# 라이브러리 import
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from dateutil import parser
import pandas as pd
from sqlalchemy import create_engine, text

In [ ]:
# MariaDB에 연결하기
db_id = ''
db_pw = ''
db_ip = ''
db_port = 0
db_name = ''

engine = create_engine(f"mysql+pymysql://{db_id}:{db_pw}@{db_ip}:{db_port}/{db_name}?charset=utf8mb4")
conn = engine.connect()

In [7]:
# 크롤링 함수
def crawler(company_code, start_date, end_date):
    page = 1
    df_result = None

    while True:
        # URL 생성
        url = f'https://finance.naver.com/item/news_news.nhn?code={company_code}&page={page}'
        # HTML 소스 가져오기
        try:
            response = requests.get(url)
            response.raise_for_status()  # HTTP 오류 발생 시 예외 처리
            source_code = response.text
        except requests.exceptions.HTTPError as e:
            print(f"[{company_code}] HTTP 오류 ({e.response.status_code}): {e}")
            return pd.DataFrame()  # 데이터 수집 실패 시 빈 DataFrame 반환
        except requests.exceptions.ConnectionError:
            print(f"[{company_code}] 연결 오류 발생 - 페이지 {page}")
            continue

        # BeautifulSoup 객체 생성
        html = BeautifulSoup(source_code, "lxml")

        # 기사 제목, 링크, 날짜 추출
        titles = html.select('.title')
        title_result = [title.get_text(strip=True) for title in titles]

        links = html.select('.title')
        link_result = ['https://finance.naver.com' + link.find('a')['href'] for link in links]

        dates = html.select('.date')
        date_result = [date.get_text(strip=True) for date in dates]

        # 문자열 날짜를 datetime 객체로 변환
        converted_dates = []
        for date_str in date_result:
            try:
                converted_date = parser.parse(date_str).date()
                converted_dates.append(converted_date)
            except ValueError:
                print(f"[{company_code}] 날짜 변환 오류: {date_str}")

        # 결과 DataFrame 생성
        result = {"dates": converted_dates, "title": title_result, "link": link_result}
        df_page = pd.DataFrame(result)

        # 날짜 형식 변환 및 비교
        df_page['dates'] = pd.to_datetime(df_page['dates'])
        df_page['dates'] = df_page['dates'].dt.date

        df_page = df_page[(df_page['dates'] >= start_date) & (df_page['dates'] <= end_date)]

        # 페이지 정보 출력 및 다음 페이지 진행
        if not df_page.empty:
            print(f"[{company_code}] 페이지 {page} 다운로드 완료")
            page += 1
        else:
            print(f"[{company_code}] 더 이상 뉴스가 없습니다.")
            break

        # 결과 DataFrame 누적
        if df_result is None:
            df_result = df_page
        else:
            df_result = pd.concat([df_result, df_page], ignore_index=True)

    return df_result


In [10]:
def main():
    # 종목 코드 목록 읽어오기
    df = ['005490','247540']
    # ['종목코드'].tolist()
    company_codes = df
    codes = list(map(lambda x: str(x).zfill(6), company_codes))

    # 종료 날짜 설정
    now = datetime.now()
    formatted_now = now.strftime('%Y%m%d')
    end_date = datetime.strptime(formatted_now.replace('.', ''), '%Y%m%d').date()

    # 시작 날짜 설정 (1년 6개월 전)
    # start_date = end_date - timedelta(days=int(1.8*365/2))
    start_date = end_date - timedelta(days=90)
    # 저장 경로 설정
    save_dir = './종목별_뉴스'
    os.makedirs(save_dir, exist_ok=True)

    # 로그 파일 설정
    log_file = open(os.path.join(save_dir, 'log.txt'), 'w')

    # 종목별 뉴스 데이터 크롤링 및 저장
    for company_code in codes:
        print(f"\n{company_code} 종목 뉴스 크롤링 시작...")
        df_temp = crawler(company_code, start_date, end_date)

        # 빈 DataFrame 처리
        if df_temp is None:
            print(f"[{company_code}] 크롤링 데이터 없음")
            continue

        # 저장 파일명 생성
        save_file = os.path.join(save_dir, f'{company_code}_뉴스_{start_date:%Y%m%d}_{end_date:%Y%m%d}.csv')

        # 데이터 저장
        try:
            df_temp.insert(1,'ticker', company_code)
            df_temp['ticker'] = df_temp['ticker'].apply(lambda x :'A' + str(x))
            df_temp['dates'] = df_temp['dates'].apply(lambda x : datetime.strftime(x , '%y-%m-%d'))
            df_temp.to_sql('news', conn, if_exists='append', index=False)
            # df_temp.info()
            conn.commit()
            df_temp.to_csv(save_file, index=False)
            print(f"[{company_code}] 뉴스 데이터 저장 완료: {save_file}")
        except Exception as e:
            print(f"[{company_code}] 데이터 저장 오류: {e}")
            log_file.write(f"{company_code}: {e}\n")

    # 로그 파일 닫기
    log_file.close()
    conn.close()
    print("\n모든 종목의 뉴스 데이터 크롤링 및 저장 완료!")

In [11]:
if __name__ == "__main__":
    main()


005490 종목 뉴스 크롤링 시작...
[005490] 페이지 1 다운로드 완료
[005490] 페이지 2 다운로드 완료
[005490] 페이지 3 다운로드 완료
[005490] 페이지 4 다운로드 완료
[005490] 페이지 5 다운로드 완료
[005490] 페이지 6 다운로드 완료
[005490] 페이지 7 다운로드 완료
[005490] 페이지 8 다운로드 완료
[005490] 페이지 9 다운로드 완료
[005490] 페이지 10 다운로드 완료
[005490] 페이지 11 다운로드 완료
[005490] 페이지 12 다운로드 완료
[005490] 페이지 13 다운로드 완료
[005490] 페이지 14 다운로드 완료
[005490] 페이지 15 다운로드 완료
[005490] 페이지 16 다운로드 완료
[005490] 페이지 17 다운로드 완료
[005490] 페이지 18 다운로드 완료
[005490] 페이지 19 다운로드 완료
[005490] 페이지 20 다운로드 완료
[005490] 페이지 21 다운로드 완료
[005490] 페이지 22 다운로드 완료
[005490] 페이지 23 다운로드 완료
[005490] 페이지 24 다운로드 완료
[005490] 페이지 25 다운로드 완료
[005490] 페이지 26 다운로드 완료
[005490] 페이지 27 다운로드 완료
[005490] 페이지 28 다운로드 완료
[005490] 페이지 29 다운로드 완료
[005490] 페이지 30 다운로드 완료
[005490] 페이지 31 다운로드 완료
[005490] 페이지 32 다운로드 완료
[005490] 페이지 33 다운로드 완료
[005490] 페이지 34 다운로드 완료
[005490] 페이지 35 다운로드 완료
[005490] 페이지 36 다운로드 완료
[005490] 페이지 37 다운로드 완료
[005490] 페이지 38 다운로드 완료
[005490] 페이지 39 다운로드 완료
[005490] 페이지 40 다운로드 완료
[005490] 페이지 41 다운로드 완료
[